# <span style="color:rgb(106,127,16)">Bikes on Tilikum Crossing</span>


<div style="text-align: right"><span style="color:rgb(106,127,16)">May 1,  2020
    </span></div>
<br>





A car-free bridge is still considered a ridiculous idea in many parts of our country (and some go so far as to call it un-American).  Portlanders beg to differ. Our newest bridge, the *Tilikum Crossing,* opened in 2015, and is highly multimodal,  allowing travel for pedestrians, bikes, electric scooters, trains, streetcars, and buses (but the modality of travel by personal car is missing). Bike lanes were not an afterthought, but rather an integral part of the bridge  design. We therefore expect to see a good amount of bike traffic on Tilikum.  


In this activity, we examine the data collected by the bicycle counters on the Tilikum.  Portland is divided into east side and west side by the north-flowing Willamette river and the Tilikum connects the two sides with both eastbound and westbound lanes. Here is a photo of the bike counter (the black display, located in between the streetcar  and the  bike lane) on the bridge.

<img src="../figs/TilikumStreecar.png" style="width:400px">

Citizens use the numbers displayed live on this little device to boast the size of our bike scene in comparison to other cities. Beyond such obvious and visible usage, once we have data, it can find more utility. The data can speak in many ways, at times  in unanticipated ways -  it is more fun than a Sherlock Holmes tale. Please feel free to put on airs like Mr Holmes while you wrangle with the data from these bike counters to eventually make the following discoveries:


- Most of those who bike to work on Tilikum live on the east side. 
    
- Recreational bikers on Tilikum prefer afternoon rides.

- There are fewer bikers on the bridge after social distancing in general who appear to use the bridge recreationally during afternoons.

Comparison with Seattle: 

- There are fewer bikers on Portland's Tilikum than on Seattle's Fremont bridge in general.  

- During peak hours, bikers are distributed more evenly on Seattle's Fremont bridge travel lanes than on Tilikum. 

- The bike usage on both bridges have shifted to a recreational pattern after social distancing.


The [BikePed Portal](http://bikeped.trec.pdx.edu) provides some of the data collected from the counter for the public, but currently only subsampled data can be  downloaded from there.  Here we shall instead use the full raw data set collected by the counters, which is not yet publicly downloadable. I gratefully acknowledge Dr. Tammy Lee and [TREC](https://trec.pdx.edu/) for making this data accessible for our course.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn; seaborn.set()

## Initial examination of the data



As you have seen in previous activities, the first step in dealing with real data is wrangling with it to make it fit our tools. The case of this data is no different.  (If you haven't yet heard of Hadley Wickham's famous paper [Tidy Data](https://www.jstatsoft.org/article/view/v059i10), J. Stat. Software, I recommend you take a look. It begins with the sentence, "A huge amount of effort is spent cleaning data to get it ready for analysis $\ldots$")

In [ ]:
# metadata file (small file)
tm = pd.read_csv('../../data_external/tilikum_metadata.csv')

# data file (large file)
td = pd.read_csv('../../data_external/tilikum_data_20200427.csv')
td.head()

Looking through first few (of the over 300,000) data entries above, and then examining the meta data file contents in `tm`, we conclude that `volume`  gives the bike counts. The volume is for 15-minute intervals as seen from `measure_period`. A quick check indicates that every data  entry has a starting and ending time that conforms to a 15-minute measurement.

In [ ]:
dif = pd.to_datetime(td['end_time']) - pd.to_datetime(td['start_time']) 
(dif == dif[0]).all()

Therefore, let us rename `start_time` to just `time` and drop the redundant data in `end_time` and `measure_period` (as well as the `id`) columns. 

In [ ]:
td = td.rename(columns={'start_time':'time'}).drop(columns=['end_time', 'measure_period', 'id'])

The meta data also tells us to expect three detectors and  three values of `flow_detector_id`. Here are a few entries from the meta data:

In [ ]:
tm.T.loc[['detector_description', 'flow_detector_id', 'detector_make', 'detector_name', 'facility_description'], :]

Although there are three values of `flow_detector_id` listed above, one of these values never seems to appear in the data file. You can check that it does not as follows:

In [ ]:
(td.flow_detector_id==1904).sum()

Therefore, going through the meta data again, we conclude that eastbound and westbound bikes pass through the flow detectors with id-numbers 1903 and 1905, respectively. 

The next step is to reshape the data into the form of a  time series. The `start_time` seems like a good candidate for indexing a time series. But it's  a red herring.  A closer look will tell you that the times are repeated in the data set. This is because there are distinct data entries for the eastbound and westbound volumes with the same time stamp. So we will make two data sets (since our data is not gigabytes long,  memory will not be an issue),  a `tE` for eastbound volume and `tW` for westbound volume.  

In [ ]:
tE = td.loc[td['flow_detector_id']==1903, ['time', 'volume']]
tE.index = pd.DatetimeIndex(pd.to_datetime(tE['time'])).tz_convert('US/Pacific')
tE = tE.drop(columns=['time']).rename(columns={'volume':'Eastbound'})

In [ ]:
tW = td.loc[td['flow_detector_id']==1905, ['time', 'volume']]
tW.index = pd.DatetimeIndex(pd.to_datetime(tW['time'])).tz_convert('US/Pacific')
tW = tW.drop(columns=['time']).rename(columns={'volume':'Westbound'})

Note that we have now indexed eastbound and westbound data by time stamps, and  renamed `volume` to `Eastbound` and `Westbound` respectively in each case. 


We are now ready for a first look at the full time series. Let us consider the eastbound data first.

In [ ]:
tE.plot();

Clearly, we have problems with this data. A spike of 7000 bikers passing through in 15 minutes, even for a bike-crazed city like Portland, just does not seem right. Zooming in, we find the situation even more disturbing, with a lot of zero readings before the spike: 

In [ ]:
tE['2018-11-25':'2019-06-01'].plot();

There are reports from [TriMet](https://trimet.org) of construction in 2018 and [city traffic advisories](https://www.portlandoregon.gov/transportation/article/728559) in 2019 that might all affect bike counter operation, but since the data set seems to have no means to indicate these outages,  we are forced to come up with some strategy ourselves for discarding the false-looking entries from the data. 


First,  exploiting `pandas`' ability to work with missing values, we declare the entries for the dates in the above plot to be missing. Note that missing data is not the same as zero data. When the bike counter is not working, the data should ideally be marked as missing, not zero. Since our suspicion is that outages might have resulted in defective counts, we shall effectively remove all data entries for these dates from the data set, as follows:

In [ ]:
tE['2018-11-25':'2019-06-01'] = np.nan

Next, we shall declare all entries with a volume of more than 1000 bikes per 15 minute to be a missing/defective value on both the westbound and eastbound data. 

In [ ]:
tE[tE > 1000] = np.nan
tW[tW > 1000] = np.nan

## Visualize cleaned up data

After the preparations above, we are now ready to visualize. Let us merge the east and west two data sets on the same time stamp axis. 

In [ ]:
t = pd.merge(tE, tW, on='time')
t.plot(alpha=0.7, style=['-',':']);

Examining the above graph, we still see spikes that look unreasonably high in the beginning of the data, but they may actually be real because  at the official [opening of the bridge](https://www.oregonlive.com/multimedia/2015/08/tilikum_crossing_public_treate.html)  there were 30,000 to 40,000 people and at least 13,000 bikes milling around. Similarly, the other spikes may be real data. One can try to explain them, e.g.,  by consulting https://bikeportland.org/events/, from which you might conclude that the spike on August 25, 2019 is due to a [Green Loop](https://bikeportland.org/event/sunday-parkways-green-loop-downtown) event, and the spike on June 29, 2019 might be due to all the people coming over for the World Naked Bike Ride. We'll just leave it at that, and blame the remaining spikes on the groovy bike scene of Portland.


The quarter-hour samples look too dense in the plot. A better picture of the situation is obtained by extracting weekly counts of bikes in both directions from the data.

In [ ]:
t.resample('W').sum().plot(style=['-',':'], title='Weekly bike counts on Tilikum');

## The pattern of use 

The Tilikum is being used both by people who commute to work using a bicyle as well as recreational bicycle users. We can understand more about this division among bikers by dividing the data into weekend and weekday entries.


The only technical skills you need for this are `numpy.where` and an understanding of `pandas.Timestamp` objects.  (Please make sure you have studied [Working with Time Series](https://jakevdp.github.io/PythonDataScienceHandbook/03.11-working-with-time-series.html) section of your textbook, which is the motivation for this activity.)  Then Combined with a use of `pandas.groupby`, we can then extract the mean biker volumes for each 15-minute interval during the day. 

The result is the distribution plotted below. 

In [ ]:
def weekplot(d, onlyweekend=False, title=None):
    weekend = np.where(d.index.weekday < 5, 'Weekday', 'Weekend')
    by_time = d.groupby([weekend, d.index.time]).mean()
    if onlyweekend: 
        if title is None: title = 'Bikes per 15-min during weekends'
        by_time.loc['Weekend'].plot(title=title)
    else:
        if title is None: title = 'Bikes per 15-min during weekdays'
        by_time.loc['Weekday'].plot(title=title)
        
weekplot(t)

The hourly distribution is distinctly "bimodal". There is a group of westbound commuters (on their way to work) on the bridge in the morning, and a group (probably the same people) traveling eastbound after work. If you look closely, you will find that there are two slightly smaller bumps indicating that there are some (although many fewer) eastbound morning bikers and westbound evening bikers across the bridge. Yet, on the whole, the data leads us to the interesting conclusion that the overwhelming majority of the **bike commuters on the Tilikum live on the east side** and  commute to the west for work and return daily.

Often the  purpose of understanding data is to guide policy and action. What might one do with the pattern we have just discovered? With the current numbers, we do not have a bike traffic problem. But if bike counts grow significantly, perhaps the city might want to consider dynamic switching of bike and pedestrian lanes on either side, or some other traffic solution, that take into account the biker commuting patterns. The knowledge of the patterns, gained from the data, should dictate a solution.

Next, let us look at the non-commuter, recreational, use in the weekends. This distribution has just one peak in sharp contrast to the  weekday distribution.

In [ ]:
weekplot(t, onlyweekend=True)

Both the eastbound and westbound lanes seem to find a good amount of use in the weekend. There is, most definitely, a preference for recreational riding in the afternoon. I suppose that is not a major surprise in Portland as afternoons are most often when we are given a reprieve from  the battleship gray of the cloud cover.

## Changes due to isolation

As you know, on March 18, 2020, in-person instructional activities at all universities in Oregon were suspended, and  on March 23 our governor issued the ["Stay Home, Save Lives" executive order](https://govsite-assets.s3.amazonaws.com/jkAULYKcSh6DoDF8wBM0_EO%2020-12.pdf). Since the  Tilikum is near two major universities in Portland, we expect the weekday bike traffic to be impacted by these measures. Let us examine what the data tells us. 

In [ ]:
weekplot(t.loc[:'2020-03-17'], title='Before social distancing')

In [ ]:
weekplot(t.loc['2020-03-17':], title='After social distancing')

Clearly, the strong bimodal distribution has weakened considerably after we all started isolating ourselves. This is no surprise, since both universities on the west side of Tilikum  have switched to remote classes. It makes sense that there are few westbound commuters in the morning.  What about the afternoon peak?  One could imagine various explanations for this: people isolating themselves all morning, getting restless in the afternoon, especially with such unusually good weather we were having in April, and deciding to take their bikes out for some fresh air? Perhaps.  In any case, we can  summarize our conclusion from the data as follows:  **social distancing has changed the weekday bike use on Tilikum from a commuter to a recreational pattern.**    

Of course, we can also compare the overall statistics before and after social distancing, but the results are too blunt to point out differences like the above.  From the statistics outputs below,  we see that the average number of bikers per quarter-hour in each direction has decreased by 1 to 1.5:

In [ ]:
t.loc[:'2020-03-17'].mean() - t.loc['2020-03-17':].mean()

The data can also tell us the reduction in terms of number of bikers per week, although we should use it with caution as we don't have enough week samples after social distancing started.

In [ ]:
t.loc[:'2020-03-17'].resample('W').sum().mean() - t.loc['2020-03-17':].resample('W').sum().mean()

The westbound lane certainly seems to have suffered more reduction in traffic after social distancing, whichever way we slice it.

## Comparison with Seattle's Fremont bridge

Although Portland claims to be the [first city](https://www.smartcitypdx.com/open-data-program) in the US to adopt the open data program, Seattle's open data program is something to envy.  Seattle's Fremont bridge bike counter data, even  way back  from 2012,  is readily available for anyone to download, thanks to their open data program (at the URL below).   This data is extensively discussed in [your textbook](https://jakevdp.github.io/PythonDataScienceHandbook/03.11-working-with-time-series.html). Let's take a peek at their data.

In [ ]:
import os
import shutil
import urllib

url = "https://data.seattle.gov/api/views/65db-xm6k/rows.csv?accessType=DOWNLOAD"
f = "../../data_external/Fremont_Bridge_Bicycle_Counter.csv"

if not os.path.isdir('../../data_external/'):
    os.mkdir('../../data_external/')

if not os.path.exists(f):
    with open(f, 'wb') as fo:
        r = urllib.request.urlopen(url)
        shutil.copyfileobj(r, fo)

In [ ]:
sd = pd.read_csv(f)
sd.tail()

Let's do some quick clean up and renaming. 

In [ ]:
sd = sd.rename(columns={'Date' : 'time', 
                        'Fremont Bridge East Sidewalk' : 'East', 
                        'Fremont Bridge West Sidewalk' : 'West'})
sd.index = pd.to_datetime(sd.loc[:, 'time'])
sd = sd.drop(columns=['time', 'Fremont Bridge Total'])
sd.head()

### Volume comparison

Note that Seattle data gives counts per hour, not counts per 15-minutes like the Tilikum data. To compare the general statistics, we should resample the Tilikum to get hourly counts. 

In [ ]:
th = t.resample('H').sum()
th.describe()   # Portland's Tilikum

In [ ]:
sd.describe()  # Seattle's Fremont

The Tilikum data is spikier than Seattle's Fremont data (compare the `max`), but the average volumes (`mean`) are clearly higher in Seattle. That the volume is higher in Seattle in even more clear if we plot weekly counts on both bridges on the same axis.

In [ ]:
sw = sd.resample('W').sum()
tw = t.resample('W').sum()
fig, axs = plt.subplots(1, 2, figsize=(13, 3), sharey=True)
plt.subplots_adjust(wspace=0.05)
sw.plot(ax=axs[0], title='Fremont bridge (Seattle) bikes/week'); 
tw.plot(ax=axs[1], title='Tilikum bridge (Portland) bikes/week');

### Daily patterns

There is a striking difference in the distribution of the average number of bikes/hour during weekdays on the two bridges.

In [ ]:
weekplot(sd, title='Seattle weekday')
weekplot(th, title='Tilikum (Portland) on weekdays (Bikes/hr)')

We conclude that **during peak hours, bikers are distributed more evenly on Seattle's Fremont bridge travel lanes than on Portland's Tilikum.**

### Changes after social distancing

In [ ]:
weekplot(sd['2020-03-17':], title='Fremont (Seattle): Weekdays after social distancing'); plt.ylabel('Bikes/hour');
weekplot(th['2020-03-17':], title='Tilikum (Seattle): Weekdays after social distancing'); plt.ylabel('Bikes/hour');

Somewhat remarkably, despite all the above-seen differences, the weekday bike counts of both cities respond to social distancing in quite the same fashion.




<hr>




<span style="color:rgb(106,127,16); font-size:8pt">These materials were created by</span> [<span style="color:rgb(106,127,16); font-size:8pt">Jay Gopalakrishnan</span>](http://web.pdx.edu/~gjay/) <span style="color:rgb(106,127,16); font-size:8pt">for a sophomore course (MTH 271) offered during the Spring 2020 quarter at Portland State University, and are made available under the</span> [<span style="color:rgb(106,127,16) ; font-size:8pt">CC-BY-SA license</span>](https://creativecommons.org/licenses/by-sa/4.0/legalcode).
 




